In [1]:
## fix auto reloading
import pandas
import os
import sys
from dotenv import load_dotenv
import json
import pandas as pd
load_dotenv()
from video_selection.google_trends import get_trends
from video_selection.search_by_keyword import write_keyword_and_id
from video_selection.filter import filter_sample_videos

In [3]:
# pipeline setup
keyword_path = './input/input_keyword_list.csv' # Modified for different disease, Input the keyword list
keyword_ref = 'sleep apnea causes' # use the first keyword for comparison in each iteration. 
num_of_result = 100

In [4]:
# test for google trends
keyword_list = get_trends(keyword_path, keyword_ref, num_of_result)
print(keyword_list[:10])

['Pain', 'anxiety', 'Sleeping', 'night', 'sore throat', 'coughing', 'Stress', 'weight gain', 'dry mouth', 'acid reflux']


/Users/ZHANGRY/Codes/youtube-analytics-pipeline/video_selection/google_trends.py:109: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  avg_popularity = normalised.mean().sort_values(ascending = False).head(num_result)


In [ ]:
# test search for keyword
df_keyword = write_keyword_and_id(keyword_list, 5)
df_keyword.head(5)

In [23]:
df_keyword

,keyword,id,rank,toKeep
304,excessive yawning,-MjaUhSOf94,5,False
367,difficulty falling asleep,PmM1IH-LaVQ,3,False
394,Obstructive Syndrome,krxcVBgerHM,5,False
480,pain in front center of brain,FSPzEzNBtMg,1,False
481,pain in front center of brain,HvVa7vUSRIo,2,False
482,pain in front center of brain,1JrdO-1Wf8g,3,False
483,pain in front center of brain,TYhgbE_84MA,4,False
484,pain in front center of brain,vPje60GgHXc,5,False


In [ ]:
# test filter and sample
credential_path = "oauth_credential.json"
# just for test
df_keyword = pd.read_csv('./temp/videoID_list.csv')
sampled_videos = filter_sample_videos(df_keyword, credential_path)
sampled_videos.head(10)

In [6]:
# Final Concatnaton
os.getenv('OAUTH_CREDENTIAL')

'{"installed":{"client_id":"45152107222-al8csdiq5n470pknclhuv4a57gjr6h6r.apps.googleusercontent.com","project_id":"geometric-rock-358702","auth_uri":"https://accounts.google.com/o/oauth2/auth","token_uri":"https://oauth2.googleapis.com/token","auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs","client_secret":"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","redirect_uris":["http://localhost"]}}'